**Assignment explanation:**

An explanation of the central idea behind your final project (what is the idea?, why is it interesting? which datasets did you need to explore the idea?, how did you download them)
A walk-through of your preliminary data-analysis, addressing
What is the total size of your data? (MB, number of rows, number of variables, etc)
What is the network you will be analyzing? (number of nodes? number of links?, degree distributions, what are node attributes?, etc.)
What is the text you will be analyzing?
How will you tie networks and text together in your paper?


The core idea of our project is to map the ecosystem of Danish music festivals by looking at who plays where, how festivals cluster, and how artist descriptions change across years and genres. This lets us explore how the Danish music scene has evolved over time.

We are especially interested in how festival identities shift. For example, Roskilde originally began as a rock and pop festival, but its lineup today spans many genres. We want to see whether festivals are becoming more similar by booking the same artists, or whether they still serve different audiences and demands. We also want to identify whether there are artists who appear across multiple generations, playing both in our parents’ time and in our own.

To explore this, we are building a dataset based on the largest and oldest festivals in Denmark, including Roskilde Festival, Smukfest, Copenhell, Vig Festival, Distortion, and others. Most of the historical lineups come from festivalhistorik.dk, and any missing years will be collected from festival websites or Wikipedia.

Altogether, this gives us around 250 nodes, where each node is a specific festival in a specific year. Two nodes will be connected if they share artists.

> missing a bit more analysis...


For the text analysis, we will use artist descriptions and the official festival descriptions from 2025. We will compare how closely each festival-year matches the 2025 description, to see how festival identities develop and how their audiences may have shifted across generations.


# code for festval api

In [4]:
import requests

festivals = []

for i in range(1, 159):  # 1 to 158 inclusive
    url = f"https://api.festivalhistorik.dk/api/festivals/{i}"
    try:
        r = requests.get(url)

        if r.status_code == 200:
            data = r.json().get("data")
            if data:
                festival_slug = data.get("slug")

                cat = data.get("festival_category", {})
                general_id = data.get("id")
                festival_name = cat.get("title")

                festivals.append((general_id, festival_slug, festival_name))

                print(f"{general_id} | {festival_slug} | {festival_name}")

        else:
            print(f"{i}: no data ({r.status_code})")

    except Exception as e:
        print(f"{i}: error {e}")

print("Done.")
print(festivals)


1 | roskilde-festival-1971 | Roskilde Festival
2 | roskilde-festival-1972 | Roskilde Festival
3 | roskilde-festival-1973 | Roskilde Festival
4 | roskilde-festival-1974 | Roskilde Festival
5 | roskilde-festival-1975 | Roskilde Festival
6 | roskilde-festival-1976 | Roskilde Festival
7 | roskilde-festival-1977 | Roskilde Festival
8 | roskilde-festival-1978 | Roskilde Festival
9 | roskilde-festival-1979 | Roskilde Festival
10 | roskilde-festival-1980 | Roskilde Festival
11 | smukfest-1980 | Smukfest
12 | roskilde-festival-1981 | Roskilde Festival
13 | smukfest-1981 | Smukfest
14 | roskilde-festival-1982 | Roskilde Festival
15 | smukfest-1982 | Smukfest
16 | roskilde-festival-1983 | Roskilde Festival
17 | smukfest-1983 | Smukfest
18 | roskilde-festival-1984 | Roskilde Festival
19 | smukfest-1996 | Smukfest
20 | smukfest-1984 | Smukfest
21 | roskilde-festival-1985 | Roskilde Festival
22 | smukfest-1985 | Smukfest
23 | nibe-festival---1985 | Nibe festival
24 | roskilde-festival-1986 | Roskild

In [9]:
# Step 2: fetch artists for each festival
all_artists = []

for fest_id, fest_slug, festival_name in festivals:
    url = f"https://api.festivalhistorik.dk/api/festivals/{fest_id}/artists"
    try:
        r = requests.get(url)
        if r.status_code == 200:
            data_artists = r.json().get("data", [])
            for artist in data_artists:
                slug = artist.get("slug")
                all_artists.append((fest_slug, slug))
                print(f"{fest_slug} | {slug}")
        else:
            print(f"Error: {r.status_code}")

    except Exception as e:
        print(f"{fest_id}: error {e}")

print("Artist scraping done.")


roskilde-festival-1971 | tom-bailey
roskilde-festival-1971 | surprise
roskilde-festival-1971 | per-kofoed
roskilde-festival-1971 | phillias-fogg
roskilde-festival-1971 | dr-dopojam
roskilde-festival-1971 | vannis-aften
roskilde-festival-1971 | sunny-side-stompers
roskilde-festival-1971 | gasolin
roskilde-festival-1971 | midnight-sun
roskilde-festival-1971 | strawbs
roskilde-festival-1971 | povl-dissing
roskilde-festival-1971 | papa-bues-jazzband
roskilde-festival-1971 | stefan-grossman
roskilde-festival-1971 | stefan-grossmand
roskilde-festival-1971 | sebastian-band
roskilde-festival-1971 | c-sar
roskilde-festival-1971 | burnin-red-ivanhoe
roskilde-festival-1971 | andrew-john
roskilde-festival-1971 | alex-campbell
roskilde-festival-1971 | lousiana-hot-seven
roskilde-festival-1971 | spider-john-koerner
roskilde-festival-1971 | fessors-big-city-band
roskilde-festival-1971 | alrune-rod
roskilde-festival-1971 | per-dich
roskilde-festival-1971 | delta-blues-band
roskilde-festival-1971 | no-

In [13]:
# takes what is needed from data fetch before
all_artists = [
    (fest, artist)
    for fest, artist in all_artists
    if (fest.startswith("smukfest-")  or fest.startswith("roskilde-festival"))
]
"""or fest.startswith("northside-")"""

'or fest.startswith("northside-")'

# code for northside

In [ ]:
import re
url = "https://northside.dk/om-northside/historie/"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15"
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()
html = resp.text

years = re.findall(r'>\s*(20\d{2})\s*<', html)
unique_years = list(set(years))
artist_northside=[]
for years in unique_years:
    url_artist=f"https://northside.dk/om-northside/historie/northside-{years}"
    fest_slug = url_artist.split("/")[-1]
    resp_artist = requests.get(url_artist, headers=headers)
    if resp_artist.status_code == 200:
        resp_artist.raise_for_status()
        html_artist = resp_artist.text
        lineup = re.search(r'<p><strong>Lineup:</strong>&nbsp;(.*?)</p>', html, re.S)
        if lineup:
            artists = lineup.group(1)
        else:
            fallback = re.search(r"Line[-\s]?up:?[^<]*</?(?:strong|h\d)[^>]*>\s*:?[\s]*(.*?)(?=<h\d|<strong|</div>)", html, re.S)
            if fallback:
                artists = fallback.group(1)

        artists = re.sub(r"<.*?>", "", artists)  # fjerner tags
        artists = artists.strip()
        for artist in artists.split(", "):
            clean = artist.replace(" ", "-").replace("&amp;", "&")
            artist_northside.append((fest_slug, clean))
    else:
        print(f"Error: {r.status_code}")

## Tinderbox

In [ ]:
import re
import requests
url = "https://tinderbox.dk/plakater/"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15"
}
resp = requests.get(url, headers=headers)
resp.raise_for_status()
html = resp.text

# pattern: grab year (2015 or 2019) and the <p> that contains "Lineup:"
pattern = r'''
    <h2[^>]*>\s*TINDERBOX\s+(2015|2019)\s*</h2>   # heading with year
    \s*<p>.*?</p>                                # dates paragraph
    \s*<p>(.*?)</p>                              # lineup paragraph
'''

matches = re.findall(pattern, html, re.S | re.I | re.X)

lineups = {}
artist_tinderbox=[]

for year, lineup_html in matches:
    # remove HTML tags
    text = re.sub(r'<.*?>', '', lineup_html)
    # strip "Lineup:" or "Lineup: " at start (case-insensitive)
    text = re.sub(r'^\s*lineup:\s*', '', text, flags=re.I)
    text = text.strip()
    # split into artists if you want a list
    artists = [a.strip() for a in text.split(',') if a.strip()]
    for artist in artists:
        artist_tinderbox.append((f"tinderbox-{year}",artist))

print(artist_tinderbox)

# Nibe festival (2000-2024)

In [15]:
import requests
import re

BASE_URL = "https://www.setlist.fm/festival/{year}/nibe-festival-{year}-73d6366d.html"

# matches artist names in links like:
# <a ... href="/setlists/...">Artist</a>
# <a ... href="/setlist/...">Artist</a>
# <a ... class="artist"...>Artist</a>
ARTIST_REGEX = re.compile(r'>([^<]+)</a>')

nibe_festival_artists = []

def scrape_year(year):
    url = BASE_URL.format(year=year)
    print("Scraping:", url)

    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if r.status_code != 200:
        print("No data for year", year)
        return []

    html = r.text

    # Extract ALL <a>text</a> matches
    candidates = ARTIST_REGEX.findall(html)

    artists = []

    for name in candidates:
        name = name.strip()

        # Filter out junk entries
        if not name:
            continue
        if "Add time" in name:
            continue
        if "Add Setlist" in name:
            continue
        if "Report festival" in name:
            continue
        if "Group by" in name:
            continue
        if "Artists" in name and "(A-Z)" in name:
            continue

        # Real artist names do NOT contain ":" or "By "
        if ":" in name and "Tribute" not in name:
            # filters things like "Scheduled: Artist"
            continue

        # Skip non-artist navigation menu links
        if name in ["Home", "Festivals"]:
            continue

        # Save potential artist
        artists.append(name)

    # remove duplicate artists
    artists = list(dict.fromkeys(artists))

    return artists


# scrape every year 2000–2024
for year in range(2000, 2025):
    artists = scrape_year(year)
    for a in artists:
        nibe_festival_artists.append((f"Nibe-Festival-{year}", a))


print("\nTotal entries:", len(nibe_festival_artists))
print(nibe_festival_artists)

all_artists.extend(nibe_festival_artists)

Scraping: https://www.setlist.fm/festival/2000/nibe-festival-2000-73d6366d.html
Scraping: https://www.setlist.fm/festival/2001/nibe-festival-2001-73d6366d.html
Scraping: https://www.setlist.fm/festival/2002/nibe-festival-2002-73d6366d.html
Scraping: https://www.setlist.fm/festival/2003/nibe-festival-2003-73d6366d.html
Scraping: https://www.setlist.fm/festival/2004/nibe-festival-2004-73d6366d.html
Scraping: https://www.setlist.fm/festival/2005/nibe-festival-2005-73d6366d.html
Scraping: https://www.setlist.fm/festival/2006/nibe-festival-2006-73d6366d.html
Scraping: https://www.setlist.fm/festival/2007/nibe-festival-2007-73d6366d.html
Scraping: https://www.setlist.fm/festival/2008/nibe-festival-2008-73d6366d.html
Scraping: https://www.setlist.fm/festival/2009/nibe-festival-2009-73d6366d.html
Scraping: https://www.setlist.fm/festival/2010/nibe-festival-2010-73d6366d.html
Scraping: https://www.setlist.fm/festival/2011/nibe-festival-2011-73d6366d.html
Scraping: https://www.setlist.fm/festiva

In [23]:
vig_years = set()

for fest_slug, artist_slug in all_artists:
    if fest_slug.startswith("vig-"):
        year = fest_slug.split("-")[-1]
        vig_years.add(year)

print(sorted(vig_years))


['1998', '2000', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2024']


# Setlist.fm festivals

In [16]:
#general for getting festival year links from the page https://www.setlist.fm/festival/
import requests
import re

def scrape_year(url):
    print("Scraping:", url)

    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if r.status_code != 200:
        print("No data for year", year)
        return []

    html = r.text

    # Extract ALL <a>text</a> matches
    candidates = ARTIST_REGEX.findall(html)

    artists = []

    for name in candidates:
        name = name.strip()

        # Filter out junk entries
        if not name:
            continue
        if "Add time" in name:
            continue
        if "Add Setlist" in name:
            continue
        if "Report festival" in name:
            continue
        if "Group by" in name:
            continue
        if "Artists" in name and "(A-Z)" in name:
            continue

        # Real artist names do NOT contain ":" or "By "
        if ":" in name and "Tribute" not in name:
            # filters things like "Scheduled: Artist"
            continue

        # Skip non-artist navigation menu links
        if name in ["Home", "Festivals"]:
            continue

        # Save potential artist
        artists.append(name)

    # remove duplicate artists
    artists = list(dict.fromkeys(artists))

    return artists

## Jelling Musikfestival

In [20]:
INDEX_URL_page1 = "https://www.setlist.fm/festivals/jelling-musikfestival-7bd6ba5c.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/jelling-musikfestival-7bd6ba5c.html?page=2"

Jelling_festival_artists = []

def get_year_links_jelling(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
        r'href="(\.\./festival/\d+/jelling-musikfestival-[^"]+)"'
    )
    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links

for links in get_year_links_jelling(INDEX_URL_page2):
    print(links)
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        Jelling_festival_artists.append((f"Jelling-Musikfestival-{year}", a_clean))

for links in get_year_links_jelling(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        Jelling_festival_artists.append((f"Jelling-Musikfestival-{year}", a_clean))

print("\nTotal entries:", len(Jelling_festival_artists))
print(Jelling_festival_artists)

all_artists.extend(Jelling_festival_artists)

https://www.setlist.fm/festival/2003/jelling-musikfestival-2003-2bd63cf2.html
Scraping: https://www.setlist.fm/festival/2003/jelling-musikfestival-2003-2bd63cf2.html
https://www.setlist.fm/festival/2002/jelling-musikfestival-2002-23d63cf3.html
Scraping: https://www.setlist.fm/festival/2002/jelling-musikfestival-2002-23d63cf3.html
https://www.setlist.fm/festival/2001/jelling-musikfestival-2001-3bd63cfc.html
Scraping: https://www.setlist.fm/festival/2001/jelling-musikfestival-2001-3bd63cfc.html
https://www.setlist.fm/festival/2000/jelling-musikfestival-2000-3d609bf.html
Scraping: https://www.setlist.fm/festival/2000/jelling-musikfestival-2000-3d609bf.html
https://www.setlist.fm/festival/1999/jelling-musikfestival-1999-1bd609b4.html
Scraping: https://www.setlist.fm/festival/1999/jelling-musikfestival-1999-1bd609b4.html
https://www.setlist.fm/festival/1998/jelling-musikfestival-1998-33d63cfd.html
Scraping: https://www.setlist.fm/festival/1998/jelling-musikfestival-1998-33d63cfd.html
https:

## Nibe Musikfestival

In [ ]:

INDEX_URL_page1 = "https://www.setlist.fm/festivals/nibe-festival-5bd68b10.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/nibe-festival-5bd68b10.html?page=2"

Nibe_festival_artists = []
def get_year_links_nibe(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
        r'href="(\.\./festival/\d+/nibe-festival-[^"]+)"'
    )
    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links

for links in get_year_links_nibe(INDEX_URL_page2):
    year = links.split("/")[-2]
    print(links)
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        Nibe_festival_artists.append((f"Nibe-Festival-{year}", a_clean))

for links in get_year_links_nibe(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        Nibe_festival_artists.append((f"Nibe-Festival-{year}", a_clean))

print("\nTotal entries:", len(Nibe_festival_artists))
print(Nibe_festival_artists)

https://www.setlist.fm/festival/2002/nibe-festival-2002-23d79c43.html
Scraping: https://www.setlist.fm/festival/2002/nibe-festival-2002-23d79c43.html
https://www.setlist.fm/festival/2001/nibe-festival-2001-6bd6366e.html
Scraping: https://www.setlist.fm/festival/2001/nibe-festival-2001-6bd6366e.html
https://www.setlist.fm/festival/2000/nibe-festival-2000-73d6366d.html
Scraping: https://www.setlist.fm/festival/2000/nibe-festival-2000-73d6366d.html
Scraping: https://www.setlist.fm/festival/2024/nibe-festival-2024-23d5ac43.html
Scraping: https://www.setlist.fm/festival/2023/nibe-festival-2023-bd40dce.html
Scraping: https://www.setlist.fm/festival/2022/nibe-festival-2022-33d4e809.html
Scraping: https://www.setlist.fm/festival/2019/nibe-festival-2019-2bd718ba.html
Scraping: https://www.setlist.fm/festival/2018/nibe-festival-2018-23d704c3.html
Scraping: https://www.setlist.fm/festival/2017/nibe-festival-2017-2bd64406.html
Scraping: https://www.setlist.fm/festival/2016/nibe-festival-2016-5bd60

## Vig festival

In [ ]:
def get_year_links_vig(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
        r'href="(\.\./festival/\d+/vig-festival-[^"]+)"'
    )
    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links


INDEX_URL_page1 = "https://www.setlist.fm/festivals/vig-festival-43d69b37.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/vig-festival-43d69b37.html?page=2"

vig_festival_artists = []

for links in get_year_links_vig(INDEX_URL_page2):
    year = links.split("/")[-2]
    print(links)
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        vig_festival_artists.append((f"vig-festival-{year}", a_clean))

for links in get_year_links_vig(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        vig_festival_artists.append((f"vig-festival-{year}", a_clean))

print("\nTotal entries:", len(vig_festival_artists))
print(vig_festival_artists)
all_artists.extend(vig_festival_artists)

https://www.setlist.fm/festival/1998/vig-festival-1998-7bd7ea60.html
Scraping: https://www.setlist.fm/festival/1998/vig-festival-1998-7bd7ea60.html
Scraping: https://www.setlist.fm/festival/2024/vig-festival-2024-2bd5a436.html
Scraping: https://www.setlist.fm/festival/2019/vig-festival-2019-3d71d33.html
Scraping: https://www.setlist.fm/festival/2018/vig-festival-2018-13d71d31.html
Scraping: https://www.setlist.fm/festival/2017/vig-festival-2017-6bd646ca.html
Scraping: https://www.setlist.fm/festival/2016/vig-festival-2016-33d64869.html
Scraping: https://www.setlist.fm/festival/2015/vig-festival-2015-73d632a5.html
Scraping: https://www.setlist.fm/festival/2014/vig-festival-2014-53d6e3d5.html
Scraping: https://www.setlist.fm/festival/2013/vig-festival-2013-3d65ddf.html
Scraping: https://www.setlist.fm/festival/2012/vig-festival-2012-13d65ddd.html
Scraping: https://www.setlist.fm/festival/2011/vig-festival-2011-1bd65ddc.html
Scraping: https://www.setlist.fm/festival/2010/vig-festival-2010

## Skive festival (skive beach festival)

In [24]:
def get_year_links_vig(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
    r'href="(\.\./festival/\d+/skive[^"]+)"')

    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links


INDEX_URL_page1 = "https://www.setlist.fm/festivals/skive-festival-63d692eb.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/skive-festival-63d692eb.html?page=2"
skive_festival_artists = []

for links in get_year_links_vig(INDEX_URL_page2):
    year = links.split("/")[-2]
    print(links)
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        skive_festival_artists.append((f"skive-festival-{year}", a_clean))

for links in get_year_links_vig(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        skive_festival_artists.append((f"skive-festival-{year}", a_clean))

print("\nTotal entries:", len(skive_festival_artists))
print(skive_festival_artists)
all_artists.extend(skive_festival_artists)

https://www.setlist.fm/festival/2001/skive-beach-party-festival-2001-23d63cf7.html
Scraping: https://www.setlist.fm/festival/2001/skive-beach-party-festival-2001-23d63cf7.html
https://www.setlist.fm/festival/2000/skive-beach-party-festival-2000-1bd65d14.html
Scraping: https://www.setlist.fm/festival/2000/skive-beach-party-festival-2000-1bd65d14.html
https://www.setlist.fm/festival/1999/skive-beach-party-festival-1999-bd6213e.html
Scraping: https://www.setlist.fm/festival/1999/skive-beach-party-festival-1999-bd6213e.html
https://www.setlist.fm/festival/1998/skive-beach-party-festival-1998-7bd7a208.html
Scraping: https://www.setlist.fm/festival/1998/skive-beach-party-festival-1998-7bd7a208.html
https://www.setlist.fm/festival/1996/skive-beach-party-festival-1996-7bd6ca4c.html
Scraping: https://www.setlist.fm/festival/1996/skive-beach-party-festival-1996-7bd6ca4c.html
Scraping: https://www.setlist.fm/festival/2025/skive-festival-2025-73d5e681.html
Scraping: https://www.setlist.fm/festival

## Rock under broen

In [25]:
def get_year_links_rock(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
        r'href="(\.\./festival/\d+/rock-under-broen-[^"]+)"'
    )
    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links


INDEX_URL_page1 = "https://www.setlist.fm/festivals/rock-under-broen-4bd6ab2e.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/rock-under-broen-4bd6ab2e.html?page=2"
rock_festival_artists = []

for links in get_year_links_rock(INDEX_URL_page2):
    year = links.split("/")[-2]
    print(links)
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        rock_festival_artists.append((f"rock-under-broen-{year}", a_clean))

for links in get_year_links_rock(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        rock_festival_artists.append((f"rock-under-broen-{year}", a_clean))

print("\nTotal entries:", len(rock_festival_artists))
print(rock_festival_artists)
all_artists.extend(rock_festival_artists)

https://www.setlist.fm/festival/2003/rock-under-broen-2003-5bd70758.html
Scraping: https://www.setlist.fm/festival/2003/rock-under-broen-2003-5bd70758.html
https://www.setlist.fm/festival/2002/rock-under-broen-2002-43d7075f.html
Scraping: https://www.setlist.fm/festival/2002/rock-under-broen-2002-43d7075f.html
https://www.setlist.fm/festival/2001/rock-under-broen-2001-4bd7075e.html
Scraping: https://www.setlist.fm/festival/2001/rock-under-broen-2001-4bd7075e.html
https://www.setlist.fm/festival/2000/rock-under-broen-2000-53d7075d.html
Scraping: https://www.setlist.fm/festival/2000/rock-under-broen-2000-53d7075d.html
https://www.setlist.fm/festival/1999/rock-under-broen-1999-5bd7075c.html
Scraping: https://www.setlist.fm/festival/1999/rock-under-broen-1999-5bd7075c.html
https://www.setlist.fm/festival/1998/rock-under-broen-1998-43d70753.html
Scraping: https://www.setlist.fm/festival/1998/rock-under-broen-1998-43d70753.html
https://www.setlist.fm/festival/1997/rock-under-broen-1997-53d70

In [35]:
unique_festivals = sorted({festival for festival, artist in rock_festival_artists})


## Jazz festival Copenhagen

In [26]:
def get_year_links_jazz(url):
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    pattern = re.compile(
        r'href="(\.\./festival/\d+/copenhagen-jazz-festival-[^"]+)"'
    )
    relative_links = pattern.findall(html)

    full_links = [
        "https://www.setlist.fm/festival/" + rel.split("/festival/")[1]
        for rel in relative_links
    ]
    return full_links


INDEX_URL_page1 = "https://www.setlist.fm/festivals/copenhagen-jazz-festival-6bd696de.html"
INDEX_URL_page2 = "https://www.setlist.fm/festivals/copenhagen-jazz-festival-6bd696de.html?page=2"
jazz_festival_artists = []

for links in get_year_links_jazz(INDEX_URL_page2):
    year = links.split("/")[-2]
    print(links)
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        jazz_festival_artists.append((f"copenhagen-jazz-festival-{year}", a_clean))

for links in get_year_links_jazz(INDEX_URL_page1):
    year = links.split("/")[-2]
    artists = scrape_year(links)
    for a in artists:
        if a=='Incorrect?' or a=='API':
            continue
        a_clean = a.replace(" ", "-").replace("&amp;", "&")
        jazz_festival_artists.append((f"copenhagen-jazz-festival-{year}", a_clean))

print("\nTotal entries:", len(jazz_festival_artists))
print(jazz_festival_artists)
all_artists.extend(jazz_festival_artists)

https://www.setlist.fm/festival/2004/copenhagen-jazz-festival-2004-7bd46a90.html
Scraping: https://www.setlist.fm/festival/2004/copenhagen-jazz-festival-2004-7bd46a90.html
https://www.setlist.fm/festival/2003/copenhagen-jazz-festival-2003-33d7a061.html
Scraping: https://www.setlist.fm/festival/2003/copenhagen-jazz-festival-2003-33d7a061.html
https://www.setlist.fm/festival/2002/copenhagen-jazz-festival-2002-73d46a95.html
Scraping: https://www.setlist.fm/festival/2002/copenhagen-jazz-festival-2002-73d46a95.html
https://www.setlist.fm/festival/2001/copenhagen-jazz-festival-2001-6bd46a96.html
Scraping: https://www.setlist.fm/festival/2001/copenhagen-jazz-festival-2001-6bd46a96.html
https://www.setlist.fm/festival/2000/copenhagen-jazz-festival-2000-7bd46a94.html
Scraping: https://www.setlist.fm/festival/2000/copenhagen-jazz-festival-2000-7bd46a94.html
Scraping: https://www.setlist.fm/festival/2025/copenhagen-jazz-festival-2025-33d5c4fd.html
Scraping: https://www.setlist.fm/festival/2024/co

In [40]:
unique_festivals = sorted({festival for festival, artist in jazz_festival_artists})
unique_festivals

['copenhagen-jazz-festival-2000',
 'copenhagen-jazz-festival-2001',
 'copenhagen-jazz-festival-2002',
 'copenhagen-jazz-festival-2003',
 'copenhagen-jazz-festival-2004',
 'copenhagen-jazz-festival-2005',
 'copenhagen-jazz-festival-2006',
 'copenhagen-jazz-festival-2007',
 'copenhagen-jazz-festival-2008',
 'copenhagen-jazz-festival-2009',
 'copenhagen-jazz-festival-2010',
 'copenhagen-jazz-festival-2011',
 'copenhagen-jazz-festival-2012',
 'copenhagen-jazz-festival-2013',
 'copenhagen-jazz-festival-2014',
 'copenhagen-jazz-festival-2015',
 'copenhagen-jazz-festival-2016',
 'copenhagen-jazz-festival-2017',
 'copenhagen-jazz-festival-2018',
 'copenhagen-jazz-festival-2019',
 'copenhagen-jazz-festival-2021',
 'copenhagen-jazz-festival-2022',
 'copenhagen-jazz-festival-2023',
 'copenhagen-jazz-festival-2024',
 'copenhagen-jazz-festival-2025']

# Copenhell festival

In [27]:
#Copenhell
import requests
from bs4 import BeautifulSoup
import time
import re
import unicodedata

headers = {"User-Agent": "Mozilla/5.0 (student project)"}

def slugify(text):
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", "-", text)
    return text.strip("-")

for year in range(2010, 2026):   # adjust years as needed
    fest_slug = f"copenhell-{year}"
    #print(f"\n=== {fest_slug} ===")

    url = f"https://heavymetal.dk/koncert/copenhell-{year}"

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
    except Exception as e:
        print(f"  Could not load page: {e}")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    header = soup.find(lambda tag: tag.name in ("h2", "h3") and "Line-up" in tag.get_text())
    if not header:
        print("  No lineup section found.")
        continue

    artists = []
    for tag in header.find_next_siblings():
        if tag.name in ("h2", "h3"):
            break
        for li in tag.find_all("li"):
            a = li.find("a")
            name = (a or li).get_text(strip=True)
            if name:
                artists.append(name)

    # remove duplicates, keep order
    artists = list(dict.fromkeys(artists))

    if not artists:
        print("  No artists found.")
        continue

    print(f"  Found {len(artists)} artists.")

    for name in artists:
        artist_slug = slugify(name)
        all_artists.append((fest_slug, artist_slug))
        print(f"  {fest_slug} | {artist_slug}")

    time.sleep(1)

print("\nCopenhell scraping done.")
print(f"Total tuples in all_artists: {len(all_artists)}")


  Found 13 artists.
  copenhell-2010 | megadeth
  copenhell-2010 | deftones
  copenhell-2010 | suicidal-tendencies
  copenhell-2010 | the-dillinger-escape-plan
  copenhell-2010 | behemoth
  copenhell-2010 | napalm-death
  copenhell-2010 | hatebreed
  copenhell-2010 | danko-jones
  copenhell-2010 | the-damned-things
  copenhell-2010 | 3-inches-of-blood
  copenhell-2010 | the-psyke-project
  copenhell-2010 | supercharger
  copenhell-2010 | whitechapel
  Found 11 artists.
  copenhell-2011 | judas-priest
  copenhell-2011 | korn
  copenhell-2011 | morbid-angel
  copenhell-2011 | mayhem
  copenhell-2011 | kvelertak
  copenhell-2011 | opeth
  copenhell-2011 | anvil
  copenhell-2011 | gwar
  copenhell-2011 | all-that-remains
  copenhell-2011 | deicide
  copenhell-2011 | protest-the-hero
  Found 12 artists.
  copenhell-2012 | slayer
  copenhell-2012 | marilyn-manson
  copenhell-2012 | lamb-of-god
  copenhell-2012 | killswitch-engage
  copenhell-2012 | mastodon
  copenhell-2012 | immortal
  cope

# Grøn koncert

In [28]:
import requests
import re
import unicodedata

WIKI_API = "https://en.wikipedia.org/w/api.php"
PAGE_TITLE = "Grøn Koncert"

headers = {"User-Agent": "Mozilla/5.0 (student project)"}

def slugify(text):
    # keep æ, ø, å as valid letters
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()
    text = re.sub(r"[^a-z0-9æøå]+", "-", text)  # allow æ ø å
    return text.strip("-")

def clean_wikilinks(text):
    # [[Foo|Bar]] -> Bar, [[Foo]] -> Foo
    text = re.sub(r"\[\[([^|\]]+)\|([^]]+)\]\]", r"\2", text)
    text = re.sub(r"\[\[([^]]+)\]\]", r"\1", text)
    text = text.replace("''", "")
    # remove parentheses that contain 'band' (e.g. '(band)', '(Danish band)')
    text = re.sub(r"\s*\([^)]*band[^)]*\)", "", text, flags=re.IGNORECASE)
    return text.strip()

# --- Get raw wikitext ---
params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "main",
    "format": "json",
    "titles": PAGE_TITLE,
}

r = requests.get(WIKI_API, params=params, headers=headers, timeout=10)
r.raise_for_status()
data = r.json()

pages = data["query"]["pages"]
page = next(iter(pages.values()))
rev = page["revisions"][0]
wikitext = rev.get("slots", {}).get("main", {}).get("*", rev.get("*", ""))

# | YEAR | Dates | Host | align=center | PERFORMERS | ...
row_pattern = re.compile(
    r"\|\s*(\d{4})\s*"           # year
    r"\|\s*[^|]*"                # dates
    r"\|\s*[^|]*"                # hosts
    r"\|\s*align=center\s*\|\s*" # align=center |
    r"([^|]*)"                   # performing artists cell
)

matches = row_pattern.findall(wikitext)

for year_str, artist_cell in matches:
    artist_cell = clean_wikilinks(artist_cell)

    # split on commas and clean names
    names = [a.strip() for a in artist_cell.split(",") if a.strip()]

    fest_slug = f"groen-koncert-{year_str}"
    print(f"\n=== {fest_slug} ({len(names)} artists) ===")

    for name in names:
        artist_slug = slugify(name)
        all_artists.append((fest_slug, artist_slug))
        print(f"{fest_slug} | {artist_slug}")

print("\nDone.")
print(f"Total tuples in all_artists: {len(all_artists)}")


=== groen-koncert-1983 (1 artists) ===
groen-koncert-1983 | sneakers

=== groen-koncert-1984 (1 artists) ===
groen-koncert-1984 | nanna-luders-jensen

=== groen-koncert-1985 (3 artists) ===
groen-koncert-1985 | frede-fup
groen-koncert-1985 | michael-falch
groen-koncert-1985 | marquis-de-sade

=== groen-koncert-1986 (3 artists) ===
groen-koncert-1986 | sanne-salomonsen
groen-koncert-1986 | thomas-helmig
groen-koncert-1986 | tv-2

=== groen-koncert-1987 (3 artists) ===
groen-koncert-1987 | anne-linnet
groen-koncert-1987 | thomas-helmig
groen-koncert-1987 | one-two

=== groen-koncert-1988 (3 artists) ===
groen-koncert-1988 | gnags
groen-koncert-1988 | sanne-salomonsen
groen-koncert-1988 | lis-sørensen

=== groen-koncert-1989 (4 artists) ===
groen-koncert-1989 | hanne-boel
groen-koncert-1989 | anne-linnet
groen-koncert-1989 | sanne-salomonsen
groen-koncert-1989 | dodo-the-dodo-s

=== groen-koncert-1990 (3 artists) ===
groen-koncert-1990 | gnags
groen-koncert-1990 | thomas-helmig
groen-kon

# Rå data Northside

In [29]:
# Rå data for northside

extra = [('northside-2016', 'ARY'),
 ('northside-2016', 'Benjamin-Hav-&-Familien'),
 ('northside-2016', 'Blæst'),
 ('northside-2016', 'Cavetown'),
 ('northside-2016', 'eee-gee'),
 ('northside-2016', 'First-Aid-Kit'),
 ('northside-2016', 'Folkeklubben'),
 ('northside-2016', 'Hans-Phillip'),
 ('northside-2016', 'Gilli'),
 ('northside-2016', 'Guldimund'),
 ('northside-2016', 'JPEGMAFIA'),
 ('northside-2016', 'KALEO'),
 ('northside-2016', 'Kevin-Morby'),
 ('northside-2016', 'Kind-mod-Kind'),
 ('northside-2016', 'Lamin'),
 ('northside-2016', 'Little-Simz'),
 ('northside-2016', 'LP'),
 ('northside-2016', 'Lukas-Graham'),
 ('northside-2016', 'Madrugada'),
 ('northside-2016', 'Muse'),
 ('northside-2016', 'Nik-&-Jay'),
 ('northside-2016', 'Nothing-But-Thieves'),
 ('northside-2016', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2016', 'Phlake'),
 ('northside-2016', 'Pil'),
 ('northside-2016', 'PRISMA'),
 ('northside-2016', 'Pusha-T'),
 ('northside-2016', 'Sko/Torp'),
 ('northside-2016', 'The-1975'),
 ('northside-2016', 'The-Chemical-Brothers'),
 ('northside-2016', 'Tobias-Rahim'),
 ('northside-2016', 'White-Lies'),
 ('northside-2016', 'Yemi-Alade'),
 ('northside-2016', 'Zar-Paulo'),
 ('northside-2013', 'ARY'),
 ('northside-2013', 'Benjamin-Hav-&-Familien'),
 ('northside-2013', 'Blæst'),
 ('northside-2013', 'Cavetown'),
 ('northside-2013', 'eee-gee'),
 ('northside-2013', 'First-Aid-Kit'),
 ('northside-2013', 'Folkeklubben'),
 ('northside-2013', 'Hans-Phillip'),
 ('northside-2013', 'Gilli'),
 ('northside-2013', 'Guldimund'),
 ('northside-2013', 'JPEGMAFIA'),
 ('northside-2013', 'KALEO'),
 ('northside-2013', 'Kevin-Morby'),
 ('northside-2013', 'Kind-mod-Kind'),
 ('northside-2013', 'Lamin'),
 ('northside-2013', 'Little-Simz'),
 ('northside-2013', 'LP'),
 ('northside-2013', 'Lukas-Graham'),
 ('northside-2013', 'Madrugada'),
 ('northside-2013', 'Muse'),
 ('northside-2013', 'Nik-&-Jay'),
 ('northside-2013', 'Nothing-But-Thieves'),
 ('northside-2013', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2013', 'Phlake'),
 ('northside-2013', 'Pil'),
 ('northside-2013', 'PRISMA'),
 ('northside-2013', 'Pusha-T'),
 ('northside-2013', 'Sko/Torp'),
 ('northside-2013', 'The-1975'),
 ('northside-2013', 'The-Chemical-Brothers'),
 ('northside-2013', 'Tobias-Rahim'),
 ('northside-2013', 'White-Lies'),
 ('northside-2013', 'Yemi-Alade'),
 ('northside-2013', 'Zar-Paulo'),
 ('northside-2015', 'ARY'),
 ('northside-2015', 'Benjamin-Hav-&-Familien'),
 ('northside-2015', 'Blæst'),
 ('northside-2015', 'Cavetown'),
 ('northside-2015', 'eee-gee'),
 ('northside-2015', 'First-Aid-Kit'),
 ('northside-2015', 'Folkeklubben'),
 ('northside-2015', 'Hans-Phillip'),
 ('northside-2015', 'Gilli'),
 ('northside-2015', 'Guldimund'),
 ('northside-2015', 'JPEGMAFIA'),
 ('northside-2015', 'KALEO'),
 ('northside-2015', 'Kevin-Morby'),
 ('northside-2015', 'Kind-mod-Kind'),
 ('northside-2015', 'Lamin'),
 ('northside-2015', 'Little-Simz'),
 ('northside-2015', 'LP'),
 ('northside-2015', 'Lukas-Graham'),
 ('northside-2015', 'Madrugada'),
 ('northside-2015', 'Muse'),
 ('northside-2015', 'Nik-&-Jay'),
 ('northside-2015', 'Nothing-But-Thieves'),
 ('northside-2015', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2015', 'Phlake'),
 ('northside-2015', 'Pil'),
 ('northside-2015', 'PRISMA'),
 ('northside-2015', 'Pusha-T'),
 ('northside-2015', 'Sko/Torp'),
 ('northside-2015', 'The-1975'),
 ('northside-2015', 'The-Chemical-Brothers'),
 ('northside-2015', 'Tobias-Rahim'),
 ('northside-2015', 'White-Lies'),
 ('northside-2015', 'Yemi-Alade'),
 ('northside-2015', 'Zar-Paulo'),
 ('northside-2023', 'ARY'),
 ('northside-2023', 'Benjamin-Hav-&-Familien'),
 ('northside-2023', 'Blæst'),
 ('northside-2023', 'Cavetown'),
 ('northside-2023', 'eee-gee'),
 ('northside-2023', 'First-Aid-Kit'),
 ('northside-2023', 'Folkeklubben'),
 ('northside-2023', 'Hans-Phillip'),
 ('northside-2023', 'Gilli'),
 ('northside-2023', 'Guldimund'),
 ('northside-2023', 'JPEGMAFIA'),
 ('northside-2023', 'KALEO'),
 ('northside-2023', 'Kevin-Morby'),
 ('northside-2023', 'Kind-mod-Kind'),
 ('northside-2023', 'Lamin'),
 ('northside-2023', 'Little-Simz'),
 ('northside-2023', 'LP'),
 ('northside-2023', 'Lukas-Graham'),
 ('northside-2023', 'Madrugada'),
 ('northside-2023', 'Muse'),
 ('northside-2023', 'Nik-&-Jay'),
 ('northside-2023', 'Nothing-But-Thieves'),
 ('northside-2023', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2023', 'Phlake'),
 ('northside-2023', 'Pil'),
 ('northside-2023', 'PRISMA'),
 ('northside-2023', 'Pusha-T'),
 ('northside-2023', 'Sko/Torp'),
 ('northside-2023', 'The-1975'),
 ('northside-2023', 'The-Chemical-Brothers'),
 ('northside-2023', 'Tobias-Rahim'),
 ('northside-2023', 'White-Lies'),
 ('northside-2023', 'Yemi-Alade'),
 ('northside-2023', 'Zar-Paulo'),
 ('northside-2010', 'ARY'),
 ('northside-2010', 'Benjamin-Hav-&-Familien'),
 ('northside-2010', 'Blæst'),
 ('northside-2010', 'Cavetown'),
 ('northside-2010', 'eee-gee'),
 ('northside-2010', 'First-Aid-Kit'),
 ('northside-2010', 'Folkeklubben'),
 ('northside-2010', 'Hans-Phillip'),
 ('northside-2010', 'Gilli'),
 ('northside-2010', 'Guldimund'),
 ('northside-2010', 'JPEGMAFIA'),
 ('northside-2010', 'KALEO'),
 ('northside-2010', 'Kevin-Morby'),
 ('northside-2010', 'Kind-mod-Kind'),
 ('northside-2010', 'Lamin'),
 ('northside-2010', 'Little-Simz'),
 ('northside-2010', 'LP'),
 ('northside-2010', 'Lukas-Graham'),
 ('northside-2010', 'Madrugada'),
 ('northside-2010', 'Muse'),
 ('northside-2010', 'Nik-&-Jay'),
 ('northside-2010', 'Nothing-But-Thieves'),
 ('northside-2010', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2010', 'Phlake'),
 ('northside-2010', 'Pil'),
 ('northside-2010', 'PRISMA'),
 ('northside-2010', 'Pusha-T'),
 ('northside-2010', 'Sko/Torp'),
 ('northside-2010', 'The-1975'),
 ('northside-2010', 'The-Chemical-Brothers'),
 ('northside-2010', 'Tobias-Rahim'),
 ('northside-2010', 'White-Lies'),
 ('northside-2010', 'Yemi-Alade'),
 ('northside-2010', 'Zar-Paulo'),
 ('northside-2017', 'ARY'),
 ('northside-2017', 'Benjamin-Hav-&-Familien'),
 ('northside-2017', 'Blæst'),
 ('northside-2017', 'Cavetown'),
 ('northside-2017', 'eee-gee'),
 ('northside-2017', 'First-Aid-Kit'),
 ('northside-2017', 'Folkeklubben'),
 ('northside-2017', 'Hans-Phillip'),
 ('northside-2017', 'Gilli'),
 ('northside-2017', 'Guldimund'),
 ('northside-2017', 'JPEGMAFIA'),
 ('northside-2017', 'KALEO'),
 ('northside-2017', 'Kevin-Morby'),
 ('northside-2017', 'Kind-mod-Kind'),
 ('northside-2017', 'Lamin'),
 ('northside-2017', 'Little-Simz'),
 ('northside-2017', 'LP'),
 ('northside-2017', 'Lukas-Graham'),
 ('northside-2017', 'Madrugada'),
 ('northside-2017', 'Muse'),
 ('northside-2017', 'Nik-&-Jay'),
 ('northside-2017', 'Nothing-But-Thieves'),
 ('northside-2017', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2017', 'Phlake'),
 ('northside-2017', 'Pil'),
 ('northside-2017', 'PRISMA'),
 ('northside-2017', 'Pusha-T'),
 ('northside-2017', 'Sko/Torp'),
 ('northside-2017', 'The-1975'),
 ('northside-2017', 'The-Chemical-Brothers'),
 ('northside-2017', 'Tobias-Rahim'),
 ('northside-2017', 'White-Lies'),
 ('northside-2017', 'Yemi-Alade'),
 ('northside-2017', 'Zar-Paulo'),
 ('northside-2012', 'ARY'),
 ('northside-2012', 'Benjamin-Hav-&-Familien'),
 ('northside-2012', 'Blæst'),
 ('northside-2012', 'Cavetown'),
 ('northside-2012', 'eee-gee'),
 ('northside-2012', 'First-Aid-Kit'),
 ('northside-2012', 'Folkeklubben'),
 ('northside-2012', 'Hans-Phillip'),
 ('northside-2012', 'Gilli'),
 ('northside-2012', 'Guldimund'),
 ('northside-2012', 'JPEGMAFIA'),
 ('northside-2012', 'KALEO'),
 ('northside-2012', 'Kevin-Morby'),
 ('northside-2012', 'Kind-mod-Kind'),
 ('northside-2012', 'Lamin'),
 ('northside-2012', 'Little-Simz'),
 ('northside-2012', 'LP'),
 ('northside-2012', 'Lukas-Graham'),
 ('northside-2012', 'Madrugada'),
 ('northside-2012', 'Muse'),
 ('northside-2012', 'Nik-&-Jay'),
 ('northside-2012', 'Nothing-But-Thieves'),
 ('northside-2012', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2012', 'Phlake'),
 ('northside-2012', 'Pil'),
 ('northside-2012', 'PRISMA'),
 ('northside-2012', 'Pusha-T'),
 ('northside-2012', 'Sko/Torp'),
 ('northside-2012', 'The-1975'),
 ('northside-2012', 'The-Chemical-Brothers'),
 ('northside-2012', 'Tobias-Rahim'),
 ('northside-2012', 'White-Lies'),
 ('northside-2012', 'Yemi-Alade'),
 ('northside-2012', 'Zar-Paulo'),
 ('northside-2019', 'ARY'),
 ('northside-2019', 'Benjamin-Hav-&-Familien'),
 ('northside-2019', 'Blæst'),
 ('northside-2019', 'Cavetown'),
 ('northside-2019', 'eee-gee'),
 ('northside-2019', 'First-Aid-Kit'),
 ('northside-2019', 'Folkeklubben'),
 ('northside-2019', 'Hans-Phillip'),
 ('northside-2019', 'Gilli'),
 ('northside-2019', 'Guldimund'),
 ('northside-2019', 'JPEGMAFIA'),
 ('northside-2019', 'KALEO'),
 ('northside-2019', 'Kevin-Morby'),
 ('northside-2019', 'Kind-mod-Kind'),
 ('northside-2019', 'Lamin'),
 ('northside-2019', 'Little-Simz'),
 ('northside-2019', 'LP'),
 ('northside-2019', 'Lukas-Graham'),
 ('northside-2019', 'Madrugada'),
 ('northside-2019', 'Muse'),
 ('northside-2019', 'Nik-&-Jay'),
 ('northside-2019', 'Nothing-But-Thieves'),
 ('northside-2019', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2019', 'Phlake'),
 ('northside-2019', 'Pil'),
 ('northside-2019', 'PRISMA'),
 ('northside-2019', 'Pusha-T'),
 ('northside-2019', 'Sko/Torp'),
 ('northside-2019', 'The-1975'),
 ('northside-2019', 'The-Chemical-Brothers'),
 ('northside-2019', 'Tobias-Rahim'),
 ('northside-2019', 'White-Lies'),
 ('northside-2019', 'Yemi-Alade'),
 ('northside-2019', 'Zar-Paulo'),
 ('northside-2018', 'ARY'),
 ('northside-2018', 'Benjamin-Hav-&-Familien'),
 ('northside-2018', 'Blæst'),
 ('northside-2018', 'Cavetown'),
 ('northside-2018', 'eee-gee'),
 ('northside-2018', 'First-Aid-Kit'),
 ('northside-2018', 'Folkeklubben'),
 ('northside-2018', 'Hans-Phillip'),
 ('northside-2018', 'Gilli'),
 ('northside-2018', 'Guldimund'),
 ('northside-2018', 'JPEGMAFIA'),
 ('northside-2018', 'KALEO'),
 ('northside-2018', 'Kevin-Morby'),
 ('northside-2018', 'Kind-mod-Kind'),
 ('northside-2018', 'Lamin'),
 ('northside-2018', 'Little-Simz'),
 ('northside-2018', 'LP'),
 ('northside-2018', 'Lukas-Graham'),
 ('northside-2018', 'Madrugada'),
 ('northside-2018', 'Muse'),
 ('northside-2018', 'Nik-&-Jay'),
 ('northside-2018', 'Nothing-But-Thieves'),
 ('northside-2018', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2018', 'Phlake'),
 ('northside-2018', 'Pil'),
 ('northside-2018', 'PRISMA'),
 ('northside-2018', 'Pusha-T'),
 ('northside-2018', 'Sko/Torp'),
 ('northside-2018', 'The-1975'),
 ('northside-2018', 'The-Chemical-Brothers'),
 ('northside-2018', 'Tobias-Rahim'),
 ('northside-2018', 'White-Lies'),
 ('northside-2018', 'Yemi-Alade'),
 ('northside-2018', 'Zar-Paulo'),
 ('northside-2014', 'ARY'),
 ('northside-2014', 'Benjamin-Hav-&-Familien'),
 ('northside-2014', 'Blæst'),
 ('northside-2014', 'Cavetown'),
 ('northside-2014', 'eee-gee'),
 ('northside-2014', 'First-Aid-Kit'),
 ('northside-2014', 'Folkeklubben'),
 ('northside-2014', 'Hans-Phillip'),
 ('northside-2014', 'Gilli'),
 ('northside-2014', 'Guldimund'),
 ('northside-2014', 'JPEGMAFIA'),
 ('northside-2014', 'KALEO'),
 ('northside-2014', 'Kevin-Morby'),
 ('northside-2014', 'Kind-mod-Kind'),
 ('northside-2014', 'Lamin'),
 ('northside-2014', 'Little-Simz'),
 ('northside-2014', 'LP'),
 ('northside-2014', 'Lukas-Graham'),
 ('northside-2014', 'Madrugada'),
 ('northside-2014', 'Muse'),
 ('northside-2014', 'Nik-&-Jay'),
 ('northside-2014', 'Nothing-But-Thieves'),
 ('northside-2014', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2014', 'Phlake'),
 ('northside-2014', 'Pil'),
 ('northside-2014', 'PRISMA'),
 ('northside-2014', 'Pusha-T'),
 ('northside-2014', 'Sko/Torp'),
 ('northside-2014', 'The-1975'),
 ('northside-2014', 'The-Chemical-Brothers'),
 ('northside-2014', 'Tobias-Rahim'),
 ('northside-2014', 'White-Lies'),
 ('northside-2014', 'Yemi-Alade'),
 ('northside-2014', 'Zar-Paulo'),
 ('northside-2011', 'ARY'),
 ('northside-2011', 'Benjamin-Hav-&-Familien'),
 ('northside-2011', 'Blæst'),
 ('northside-2011', 'Cavetown'),
 ('northside-2011', 'eee-gee'),
 ('northside-2011', 'First-Aid-Kit'),
 ('northside-2011', 'Folkeklubben'),
 ('northside-2011', 'Hans-Phillip'),
 ('northside-2011', 'Gilli'),
 ('northside-2011', 'Guldimund'),
 ('northside-2011', 'JPEGMAFIA'),
 ('northside-2011', 'KALEO'),
 ('northside-2011', 'Kevin-Morby'),
 ('northside-2011', 'Kind-mod-Kind'),
 ('northside-2011', 'Lamin'),
 ('northside-2011', 'Little-Simz'),
 ('northside-2011', 'LP'),
 ('northside-2011', 'Lukas-Graham'),
 ('northside-2011', 'Madrugada'),
 ('northside-2011', 'Muse'),
 ('northside-2011', 'Nik-&-Jay'),
 ('northside-2011', 'Nothing-But-Thieves'),
 ('northside-2011', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2011', 'Phlake'),
 ('northside-2011', 'Pil'),
 ('northside-2011', 'PRISMA'),
 ('northside-2011', 'Pusha-T'),
 ('northside-2011', 'Sko/Torp'),
 ('northside-2011', 'The-1975'),
 ('northside-2011', 'The-Chemical-Brothers'),
 ('northside-2011', 'Tobias-Rahim'),
 ('northside-2011', 'White-Lies'),
 ('northside-2011', 'Yemi-Alade'),
 ('northside-2011', 'Zar-Paulo'),
 ('northside-2022', 'ARY'),
 ('northside-2022', 'Benjamin-Hav-&-Familien'),
 ('northside-2022', 'Blæst'),
 ('northside-2022', 'Cavetown'),
 ('northside-2022', 'eee-gee'),
 ('northside-2022', 'First-Aid-Kit'),
 ('northside-2022', 'Folkeklubben'),
 ('northside-2022', 'Hans-Phillip'),
 ('northside-2022', 'Gilli'),
 ('northside-2022', 'Guldimund'),
 ('northside-2022', 'JPEGMAFIA'),
 ('northside-2022', 'KALEO'),
 ('northside-2022', 'Kevin-Morby'),
 ('northside-2022', 'Kind-mod-Kind'),
 ('northside-2022', 'Lamin'),
 ('northside-2022', 'Little-Simz'),
 ('northside-2022', 'LP'),
 ('northside-2022', 'Lukas-Graham'),
 ('northside-2022', 'Madrugada'),
 ('northside-2022', 'Muse'),
 ('northside-2022', 'Nik-&-Jay'),
 ('northside-2022', 'Nothing-But-Thieves'),
 ('northside-2022', 'NxWorries-feat.&nbsp;Anderson-.Paak-&-Knxwledge'),
 ('northside-2022', 'Phlake'),
 ('northside-2022', 'Pil'),
 ('northside-2022', 'PRISMA'),
 ('northside-2022', 'Pusha-T'),
 ('northside-2022', 'Sko/Torp'),
 ('northside-2022', 'The-1975'),
 ('northside-2022', 'The-Chemical-Brothers'),
 ('northside-2022', 'Tobias-Rahim'),
 ('northside-2022', 'White-Lies'),
 ('northside-2022', 'Yemi-Alade'),
 ('northside-2022', 'Zar-Paulo')]

all_artists.extend(extra)

# Rå data Tinderbox

In [30]:
# Rå data for tinderbox
extra = [('tinderbox-2019', 'A$AP Rocky'), ('tinderbox-2019', 'Benal'), ('tinderbox-2019', 'Billie Eilish'), ('tinderbox-2019', 'The Chainsmokers'), ('tinderbox-2019', 'Crown The Beast'), ('tinderbox-2019', 'Dean Lewis'), ('tinderbox-2019', 'Den Gale Pose'), ('tinderbox-2019', 'Dermot Kennedy'), ('tinderbox-2019', 'Diplo'), ('tinderbox-2019', 'Dropkick Murphys'), ('tinderbox-2019', 'Duran Duran'), ('tinderbox-2019', 'Gilli'), ('tinderbox-2019', 'Giorgio Moroder'), ('tinderbox-2019', 'Gnags'), ('tinderbox-2019', 'Greta Van Fleet'), ('tinderbox-2019', 'J Balvin'), ('tinderbox-2019', 'Karenina'), ('tinderbox-2019', 'Khea'), ('tinderbox-2019', 'L.O.C.'), ('tinderbox-2019', 'Lana Del Rey'), ('tinderbox-2019', 'The Lumineers'), ('tinderbox-2019', 'Lydmor'), ('tinderbox-2019', 'Macklemore'), ('tinderbox-2019', 'Michael Falch'), ('tinderbox-2019', 'Miley Cyrus'), ('tinderbox-2019', 'The Minds Of 99'), ('tinderbox-2019', 'Neil Young and Promise of The Real'), ('tinderbox-2019', 'Peter Sommer &amp; Tiggerne'), ('tinderbox-2019', 'Phlake'), ('tinderbox-2019', 'Rick Astley'), ('tinderbox-2019', 'The Savage Rose'), ('tinderbox-2019', 'Sheryl Crow'), ('tinderbox-2019', 'Sigrid'), ('tinderbox-2019', 'Skunk Anansie'), ('tinderbox-2019', 'Stefflon Don'), ('tinderbox-2019', 'Suspekt'), ('tinderbox-2019', 'Swedish House Mafia'), ('tinderbox-2019', 'Trouble Is'), ('tinderbox-2019', 'Wolfmother.'), ('tinderbox-2015', 'Above &amp; Beyond'), ('tinderbox-2015', 'All Time Low'), ('tinderbox-2015', 'Angel Haze'), ('tinderbox-2015', 'Axwell Λ Ingrosso'), ('tinderbox-2015', 'Bikstok'), ('tinderbox-2015', 'Black Rebel Motorcycle Club'), ('tinderbox-2015', 'Calvin Harris'), ('tinderbox-2015', 'The Cardigans'), ('tinderbox-2015', 'Carpark North'), ('tinderbox-2015', 'D-A-D'), ('tinderbox-2015', 'De Eneste To'), ('tinderbox-2015', 'Eagles of Death Metal'), ('tinderbox-2015', 'Echosmith'), ('tinderbox-2015', 'Ellie Goulding'), ('tinderbox-2015', 'Emeli Sandé'), ('tinderbox-2015', 'Faith No More'), ('tinderbox-2015', 'Faithless'), ('tinderbox-2015', 'The Gaslight Anthem'), ('tinderbox-2015', 'Go Go Berlin'), ('tinderbox-2015', 'Hozier'), ('tinderbox-2015', 'James Blake'), ('tinderbox-2015', 'Joey Bada$$'), ('tinderbox-2015', 'Julias Moon'), ('tinderbox-2015', 'Kent'), ('tinderbox-2015', 'Lars H.U.G.'), ('tinderbox-2015', 'Love Shop'), ('tinderbox-2015', 'Major Lazer'), ('tinderbox-2015', 'Marie Key'), ('tinderbox-2015', '&nbsp;Marmozets'), ('tinderbox-2015', 'Milky Chance'), ('tinderbox-2015', 'Modest Mouse'), ('tinderbox-2015', 'MØ'), ('tinderbox-2015', 'Nothing But Thieves'), ('tinderbox-2015', 'Passenger'), ('tinderbox-2015', 'The Prodigy'), ('tinderbox-2015', 'Rasmus Walter'), ('tinderbox-2015', 'Rival Sons'), ('tinderbox-2015', 'Robbie Williams'), ('tinderbox-2015', 'Royal Blood'), ('tinderbox-2015', 'Sheppard'), ('tinderbox-2015', 'Spids Nøgenhat'), ('tinderbox-2015', 'TopGunn.')]


all_artists.extend(extra)

# Making the graph

In [33]:
all_artists = [
    (fest.lower(), artist.lower()) 
    for fest, artist in all_artists
]

In [34]:
import networkx as nx
from collections import defaultdict

# --------------------------------------------
# 1. Build mapping: festival_year -> set of artists
# --------------------------------------------
fest_to_artists = defaultdict(set)

for fest_slug, artist_slug in all_artists:
    fest_to_artists[fest_slug].add(artist_slug)

# --------------------------------------------
# 2. Create directed graph
# --------------------------------------------
G = nx.DiGraph()

# Add all nodes
for fest_slug in fest_to_artists:
    G.add_node(fest_slug)

# --------------------------------------------
# 3. Add edges based on shared artists
# --------------------------------------------
festivals = list(fest_to_artists.keys())
n = len(festivals)

for i in range(n):
    for j in range(n):
        if i == j:
            continue

        f1 = festivals[i]
        f2 = festivals[j]

        # Count shared artists
        shared = fest_to_artists[f1].intersection(fest_to_artists[f2])
        weight = len(shared)

        if weight > 0:
            G.add_edge(f1, f2, weight=weight)

print("Directed graph created.")
print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")


Directed graph created.
Nodes: 317
Edges: 29122


In [35]:
import csv

OUTPUT_FILENAME = 'festival_artist_data.csv'
HEADERS = ['Festival_Year', 'Artist_Name']

def export_to_csv(data_list, filename, headers):
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            writer.writerows(data_list)
    except:
        pass
        
export_to_csv(all_artists, OUTPUT_FILENAME, HEADERS)